In [1]:
# "US Presidential Candidate Tweets"
> "A visualization of recent tweets from candidates in the 2020 US Presidential Election"

- toc: false
- branch: master
- badges: true
- comments: true
- categories: [Data Visualization]
- image: 
- hide: false
- search_exclude: true

SyntaxError: invalid syntax (<ipython-input-1-a8be3b8d37e3>, line 2)

In [2]:
Test content.

SyntaxError: invalid syntax (<ipython-input-2-36bd102e1472>, line 1)